Setup & Konfiguration

In [1]:
!pip install rdflib pyvis


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [8]:
# --- Notebook-Header: Pipeline-Integration mit RenkuLab ---

from pathlib import Path
import pandas as pd
import json, subprocess, hashlib, sys, math, shutil
from datetime import datetime
from rdflib import Graph, Namespace, URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS
import networkx as nx
from typing import Optional, List, Set
from urllib.parse import unquote

# ----------------------- Pipeline-Pfade (RenkuLab Integration) -----------------------
# INPUT: Von RenkuLab Pipeline generierte RDF
INPUT_RDF_PATH = Path("/home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl")

# OUTPUT: Mit XMP-Daten angereicherte RDF  
OUTPUT_RDF_PATH = Path("./catalog_enriched.ttl")       # ← Finale erweiterte Version

# BACKUP: Timestamped Backup vor Verarbeitung
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
BACKUP_RDF_PATH = Path(f"./dca_catalog_backup_{timestamp}.ttl")

# TOOLS
EXIFTOOL = "/home/renku/work/exiftool/exiftool"  # ggf. absoluter Pfad anpassen
FILES_BASE_DIR = Path("/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server")

# ----------------------- Namespaces ---------------------------
DCA      = Namespace("http://dca.ethz.ch/ontology#")
DCA_ID   = Namespace("http://dca.ethz.ch/id/")
DCA_TECH = Namespace("http://dca.ethz.ch/tech#")
PREMIS   = Namespace("http://www.loc.gov/premis/rdf/v3/")
RICO     = Namespace("https://www.ica.org/standards/RiC/ontology#")
XSDNS    = XSD  # Abkürzung
# DCTERMS ist oben importiert

# ----------------------- Ziel: Bild-/Adobe-Typen -------------
IMG_EXT = {
    # klassische Bilder
    "jpg","jpeg","tif","tiff","png","gif","bmp",
    # RAW (optional – je nach Bestand)
    "dng","cr2","nef","arw"
}
ADOBE_EXT = {"psd","psb","ai","indd","pdf"}  # PDF häufig Photoshop/Illustrator-Export

TARGET_EXT = IMG_EXT | ADOBE_EXT

# === ID-Helfer basierend auf DROID CSV: hash_md5[:16] ===
from rdflib import URIRef

DCA_ID_BASE = "http://dca.ethz.ch/id/"

def dca_file_uri_from_md5(md5_hex: Optional[str]) -> Optional[URIRef]:
    """
    Erzeugt dca-id:file_<md5[:16]> aus einem MD5-Hexstring (ohne Leerzeichen).
    Gibt None zurück, wenn md5_hex leer/ungültig ist.
    """
    if not md5_hex or not isinstance(md5_hex, str):
        return None
    md5_hex = md5_hex.strip().lower()
    if len(md5_hex) < 16:
        return None
    short = md5_hex[:16]
    return URIRef(DCA_ID_BASE + f"file_{short}")

def dca_file_uri_from_path(file_path: str) -> Optional[URIRef]:
    """
    Erzeugt dca-id:file_URI basierend auf DROID MD5-Hash (bevorzugt) oder Pfad-Fallback.
    Diese Funktion ersetzt die alte pfad-basierte Implementierung.
    """
    # 1. Versuche MD5-Hash aus DROID CSV zu verwenden
    md5_hash = md5_for_abs_path(file_path)
    if md5_hash:
        return dca_file_uri_from_md5(md5_hash)
    
    # 2. Fallback: Pfad-basierter Hash (für Rückwärtskompatibilität)
    print(f"⚠️  Fallback zu Pfad-Hash für: {file_path}")
    path_hash = hashlib.sha256(file_path.encode('utf-8')).hexdigest()[:16]
    return URIRef(DCA_ID_BASE + f"file_{path_hash}")

def md5_for_abs_path(p: str) -> Optional[str]:
    """
    Gibt den MD5-Hash für einen Dateipfad zurück, falls im DROID CSV vorhanden.
    """
    return path_to_md5.get(p)

def safe_literal_dt(text: str, datatype=XSD.dateTime):
    try:
        return Literal(text, datatype=datatype)
    except Exception:
        return Literal(text)  # fall back

def run_exiftool_json(files: list, fast=False):
    """Rufe exiftool als JSON auf, tolerant gegen Minor Errors, UTF-8 Dateinamen."""
    if not files:
        return []
    cmd = [EXIFTOOL, "-a","-s","-G1","-json", "-charset","filename=UTF8","-m"]
    if fast:
        cmd.insert(1, "-fast")
    tags = [
        "XMP-xmpMM:DocumentID",
        "XMP-xmpMM:InstanceID",
        "XMP-xmpMM:OriginalDocumentID",
        "XMP-xmpMM:DerivedFromDocumentID",
        "XMP-xmpMM:DerivedFromInstanceID",
        "XMP-xmp:CreatorTool",
        "File:FileName",
        "File:Directory",
        "File:FileModifyDate",
    ]
    cmd += tags + files

    res = subprocess.run(cmd, text=True, capture_output=True)
    out = res.stdout.strip()
    try:
        return json.loads(out) if out else []
    except Exception as e:
        print("EXIF JSON parse error:", e, file=sys.stderr)
        return []

def add_identifier_triple(g: Graph, file_uri: URIRef, id_type: str, value: str):
    """Hänge einen PREMIS-Identifier als Blank Node an ein File-Objekt."""
    if not value:
        return
    bn = BNode()
    g.add((file_uri, PREMIS.hasIdentifier, bn))
    g.add((bn, PREMIS.identifierType, Literal(id_type)))
    g.add((bn, PREMIS.identifierValue, Literal(value)))

# MD5-Hash-Mapping (wird später aus RDF geladen)
path_to_md5 = {}

# ----------------------- RenkuLab Pipeline Überprüfung -----------------------
print("🚀 RenkuLab Pipeline Integration")
print(f"📂 Input RDF: {INPUT_RDF_PATH}")

if INPUT_RDF_PATH.exists():
    size_mb = INPUT_RDF_PATH.stat().st_size / 1024 / 1024
    print(f"✅ RenkuLab Output gefunden: {size_mb:.1f} MB")
else:
    print(f"❌ RenkuLab Output nicht gefunden!")
    print("   → Prüfe RenkuLab Pipeline oder Pfad")

print(f"🎯 Files Base: {FILES_BASE_DIR}")
print(f"💾 Output wird gespeichert als: {OUTPUT_RDF_PATH}")

🚀 RenkuLab Pipeline Integration
📂 Input RDF: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl
✅ RenkuLab Output gefunden: 6.9 MB
🎯 Files Base: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server
💾 Output wird gespeichert als: catalog_enriched.ttl


RDF laden, Backup erstellen & Kandidaten aus RDF extrahieren


In [4]:
# =====================================================
# PIPELINE INTEGRATION: RDF LADEN UND BACKUP ERSTELLEN
# =====================================================

# 1. Sicherheitsprüfungen
if not INPUT_RDF_PATH.exists():
    print(f"❌ Input RDF not found: {INPUT_RDF_PATH}")
    print("   → Stelle sicher, dass RenkuLab Pipeline erfolgreich war")
    sys.exit(1)

# 2. Backup der Original-RDF erstellen  
print(f"🔒 Creating backup: {BACKUP_RDF_PATH.name}")
shutil.copy2(INPUT_RDF_PATH, BACKUP_RDF_PATH)
print(f"✅ Backup created: {BACKUP_RDF_PATH.stat().st_size / 1024 / 1024:.2f} MB")

# 3. RDF Graph laden
print(f"📂 Loading RDF from: {INPUT_RDF_PATH}")
graph = Graph()
graph.parse(INPUT_RDF_PATH, format='turtle')
original_triples = len(graph)
print(f"📊 Loaded: {original_triples:,} triples")

# 4. Kandidaten aus RDF extrahieren (Bild/Adobe-Dateien)
print("🔍 Extracting image/Adobe files from RDF...")
candidate_data = []

# SPARQL Query für Kandidaten-Dateien
query = """
    PREFIX dca: <http://dca.ethz.ch/ontology#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
    
    SELECT ?file ?title ?identifier ?format WHERE {
        ?file a dca:ArchiveFile ;
              dcterms:title ?title ;
              dcterms:identifier ?identifier .
        OPTIONAL { ?file premis:hasFormatName ?format }
        
        # Filter für Bild/Adobe-Dateien über Titel-Extension
        FILTER(
            CONTAINS(LCASE(?title), ".jpg") ||
            CONTAINS(LCASE(?title), ".jpeg") ||
            CONTAINS(LCASE(?title), ".tif") ||
            CONTAINS(LCASE(?title), ".tiff") ||
            CONTAINS(LCASE(?title), ".png") ||
            CONTAINS(LCASE(?title), ".psd") ||
            CONTAINS(LCASE(?title), ".ai") ||
            CONTAINS(LCASE(?title), ".pdf")
        )
    }
"""

results = graph.query(query)
for row in results:
    file_uri = str(row.file)
    title = str(row.title)
    identifier = str(row.identifier) 
    format_name = str(row.format) if row.format else "Unknown"
    
    # Intelligentere lokale Pfad-Extraktion aus WebDAV URL
    if identifier.startswith("https://nextcloud.ethz.ch/"):
        # URL decode und lokalen Pfad konstruieren
        try:
            # Für gramazio-kohler-archiv-server URLs
            if "gramazio-kohler-archiv-server" in identifier:
                # Extrahiere alles nach dem letzten "gramazio-kohler-archiv-server/"
                parts = identifier.split("gramazio-kohler-archiv-server/")
                if len(parts) > 1:
                    path_part = unquote(parts[-1])  # Letzten Teil nehmen (nach dem Server-Namen)
                    local_path = FILES_BASE_DIR / path_part
                else:
                    # Fallback: Nur Dateiname verwenden
                    local_path = FILES_BASE_DIR / Path(unquote(identifier)).name
            else:
                # Fallback für andere NextCloud URLs
                local_path = FILES_BASE_DIR / Path(unquote(identifier)).name
        except Exception as e:
            print(f"⚠️  URL parsing failed for {identifier}: {e}")
            local_path = Path(title)  # Fallback zum Titel
    else:
        local_path = Path(title)  # Fallback
    
    candidate_data.append({
        'file_uri': file_uri,
        'title': title,
        'identifier': identifier,
        'format_name': format_name,
        'local_path': str(local_path),
        'exists': local_path.exists() if local_path.is_absolute() else False
    })

# Zu DataFrame für weitere Verarbeitung
cand = pd.DataFrame(candidate_data)
cand["ABS_PATH"] = cand["local_path"]  # Kompatibilität mit bestehendem Code

# EXT-Spalte aus Dateinamen extrahieren für Kompatibilität
cand["EXT"] = cand["title"].str.lower().str.extract(r'\.([^.]+)$')

print(f"📊 Total candidates: {len(cand):,}")
print(f"📁 Files exist locally: {cand['exists'].sum():,}")

# Debug: Zeige ein paar Beispiel-Pfade
print(f"\n🔍 Sample path mappings:")
for _, row in cand.head(3).iterrows():
    print(f"   Title: {row['title']}")
    print(f"   ID:    {row['identifier'][:50]}{'...' if len(row['identifier']) > 50 else ''}")
    print(f"   Local: {row['local_path']}")
    print(f"   Exists: {row['exists']}")
    print()

print(f"📈 Graph ready for XMP enrichment")
cand.head(3)

🔒 Creating backup: dca_catalog_backup_20260227_142830.ttl
✅ Backup created: 6.93 MB
📂 Loading RDF from: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl
📊 Loaded: 124,174 triples
🔍 Extracting image/Adobe files from RDF...
📊 Total candidates: 6,516
📁 Files exist locally: 0
📈 Graph ready for XMP enrichment


,file_uri,title,identifier,format_name,local_path,exists,ABS_PATH,EXT
0,http://dca.ethz.ch/id/file_0001615cb891ac0f,fallingSpheres_0401.tif,https://nextcloud.ethz.ch/remote.php/dav/files...,Tagged Image File Format,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,False,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,tif
1,http://dca.ethz.ch/id/file_00084414b2765927,fluid16.jpg,https://nextcloud.ethz.ch/remote.php/dav/files...,JPEG File Interchange Format,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,False,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,jpg
2,http://dca.ethz.ch/id/file_00148359ca3be9a7,fallingSpheres_1677.tif,https://nextcloud.ethz.ch/remote.php/dav/files...,Tagged Image File Format,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,False,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,tif


Neue MD5-basierte File-IDs testen

In [5]:
# =====================================================
# ID-KONSISTENZ: RDF FILE-URI zu MD5 MAPPING
# =====================================================

# Extrahiere MD5-Hash aus file_uri für Konsistenz-Check
def extract_md5_from_uri(file_uri: str) -> str:
    """Extrahiert MD5[:16] aus dca-id:file_<md5> URI"""
    if "file_" in file_uri:
        return file_uri.split("file_")[-1]
    return None

# MD5-Konsistenz prüfen
print("🔍 Checking ID consistency...")
cand["md5_from_uri"] = cand["file_uri"].apply(extract_md5_from_uri)

# Beispiel-IDs zeigen
print("\n📋 Sample ID mapping:")
for _, row in cand.head(3).iterrows():
    print(f"   {row['title']} → {row['md5_from_uri']}")

print(f"✅ {len(cand)} file URIs ready for XMP processing")

# Erstelle path_to_md5 Mapping aus der RDF (falls MD5-Hashes in PREMIS vorhanden sind)
# Das ist ein Fallback, falls keine DROID CSV verfügbar ist
path_to_md5 = {}
print(f"📊 MD5-Hash-Mapping erstellt für {len(path_to_md5)} Dateien")
print(f"   Beispiele: {list(path_to_md5.items())[:3]}")

🔍 Checking ID consistency...

📋 Sample ID mapping:
   fallingSpheres_0401.tif → 0001615cb891ac0f
   fluid16.jpg → 00084414b2765927
   fallingSpheres_1677.tif → 00148359ca3be9a7
✅ 6516 file URIs ready for XMP processing
📊 MD5-Hash-Mapping erstellt für 0 Dateien
   Beispiele: []


In [6]:
# Test der neuen MD5-basierten File-URIs
print("🧪 Testing neue MD5-basierte File-URI Generierung:\n")

# Teste mit ein paar Beispieldateien
test_files = cand["ABS_PATH"].head(5).tolist()
for file_path in test_files:
    md5_hash = md5_for_abs_path(file_path)
    old_uri = URIRef(DCA_ID_BASE + f"file_{hashlib.sha256(file_path.encode('utf-8')).hexdigest()[:16]}")
    new_uri = dca_file_uri_from_path(file_path)
    
    print(f"📁 Datei: {Path(file_path).name}")
    print(f"   MD5: {md5_hash[:16] if md5_hash else 'None'}")
    print(f"   ALT: {str(old_uri).split('/')[-1]}")
    print(f"   NEU: {str(new_uri).split('/')[-1]}")
    print(f"   ✅ Verschiedene IDs: {str(old_uri) != str(new_uri)}")
    print()

print(f"📈 Statistik:")
print(f"   Dateien mit MD5: {len([p for p in test_files if md5_for_abs_path(p)])}/{len(test_files)}")
print(f"   MD5-basierte URIs: {len([dca_file_uri_from_path(p) for p in test_files if md5_for_abs_path(p)])}")

🧪 Testing neue MD5-basierte File-URI Generierung:

⚠️  Fallback zu Pfad-Hash für: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/https:/nextcloud.ethz.ch/remote.php/dav/files/padrian/DCA/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/04_Entwurf/060730_FinalDesignDocumentation/02_Movies/01_RawMaterial/01_FallingSpheres/fallingSpheres_0401.tif
📁 Datei: fallingSpheres_0401.tif
   MD5: None
   ALT: file_a77661193869a6e7
   NEU: file_a77661193869a6e7
   ✅ Verschiedene IDs: False

⚠️  Fallback zu Pfad-Hash für: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/https:/nextcloud.ethz.ch/remote.php/dav/files/padrian/DCA/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060607_Fassadenstudie/060606_maps/fluid16.jpg
📁 Datei: fluid16.jpg
   MD5: None
   ALT: file_f635187cc02aaa

## ✨ Neue MD5-basierte File-IDs

**Wichtige Änderung**: Anstatt Pfad-basierter Hashes verwenden wir jetzt die MD5-Hashes aus dem DROID CSV:

### Vorteile:
- **Konsistent**: Gleiche Datei = Gleiche ID (unabhängig vom Pfad)  
- **Plattform-unabhängig**: Windows vs Unix Pfade irrelevant
- **Inhalt-basiert**: Nur Dateiänderungen erzeugen neue IDs
- **DROID-integriert**: Nutzt bereits berechnete Hashes

### Implementierung:
```python
# ALT: Pfad → SHA256[:16]
old_id = "file_" + hashlib.sha256(path.encode()).hexdigest()[:16]

# NEU: DROID MD5[:16] 
new_id = "file_" + droid_record.hash_md5[:16]
```

EXIF/XMP an einer Beispiel‑PSD auslesen (inkl. IDs)

In [9]:
# Eine Beispiel-PSD aus dem gefilterten DataFrame
ex_psd = cand[cand["EXT"] == "psd"].head(1)
if ex_psd.empty:
    print("Keine PSD gefunden – bitte eine PSD im Bestand wählen.")
else:
    psd_path = ex_psd["ABS_PATH"].iloc[0]
    data = run_exiftool_json([psd_path])
    print("Datei:", psd_path)
    print(json.dumps(data, indent=2))
    # Interpretation für Einsteiger (kurze Erklärung):
    print("\nHinweis:")
    print("- XMP-xmpMM:DocumentID      = Stabiler Dokument-Identifier")
    print("- XMP-xmpMM:InstanceID      = Jede Revision/Instanz hat eine neue ID")
    print("- XMP-xmpMM:DerivedFrom*    = Verknüpft auf Quelle/Parent (Ableitung)")

Datei: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/https:/nextcloud.ethz.ch/remote.php/dav/files/padrian/DCA/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060612_Fassadenstudie/model/02.psd
[]

Hinweis:
- XMP-xmpMM:DocumentID      = Stabiler Dokument-Identifier
- XMP-xmpMM:InstanceID      = Jede Revision/Instanz hat eine neue ID
- XMP-xmpMM:DerivedFrom*    = Verknüpft auf Quelle/Parent (Ableitung)


EXIF/XMP an einer Beispiel‑JPG auslesen (inkl. IDs)

In [10]:
# Eine Beispiel-JPG
ex_jpg = cand[cand["EXT"].isin(["jpg","jpeg"])].head(1)
if ex_jpg.empty:
    print("Keine JPG/JPEG gefunden – bitte ein Beispiel wählen.")
else:
    jpg_path = ex_jpg["ABS_PATH"].iloc[0]
    data = run_exiftool_json([jpg_path])
    print("Datei:", jpg_path)
    print(json.dumps(data, indent=2))
    print("\nHinweis:")
    print("- Bei JPG sind XMP-IDs nicht immer gesetzt; wenn vorhanden, nutzen wir sie wie bei PSD.")

Datei: /home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/https:/nextcloud.ethz.ch/remote.php/dav/files/padrian/DCA/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060607_Fassadenstudie/060606_maps/fluid16.jpg
[]

Hinweis:
- Bei JPG sind XMP-IDs nicht immer gesetzt; wenn vorhanden, nutzen wir sie wie bei PSD.


Batch‑Auslesen (XMP‑IDs) & Index für Derivatsuche aufbauen

In [ ]:
# =====================================================
# XMP BATCH-PROCESSING - NUR DATEIEN OHNE BESTEHENDE XMP-DATEN
# =====================================================

# 1. Prüfe, welche Dateien bereits XMP-Identifier haben
print("🔍 Checking for existing XMP data in RDF...")
existing_xmp_files = set()

# SPARQL: Finde Dateien mit XMP-Identifiern
xmp_query = """
    PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
    
    SELECT ?file WHERE {
        ?file premis:hasIdentifier ?identifier .
        ?identifier premis:identifierType ?type .
        FILTER(CONTAINS(LCASE(?type), "xmp"))
    }
"""

for row in graph.query(xmp_query):
    existing_xmp_files.add(str(row.file))

print(f"📊 Files with existing XMP: {len(existing_xmp_files)}")

# 2. Filter Kandidaten: Nur Dateien OHNE XMP-Daten und existierende lokale Dateien
needs_xmp = cand[
    (~cand['file_uri'].isin(existing_xmp_files)) &  # Keine XMP-Daten
    (cand['exists'] == True)                        # Datei existiert lokal
].copy()

print(f"📁 Files needing XMP extraction: {len(needs_xmp):,}")
print(f"⏩ Files to skip (already have XMP): {len(cand) - len(needs_xmp):,}")

# 3. Batch-Processing Setup
BATCH = 50  # Kleinere Batches für ExifTool
records = []
processed_count = 0
error_count = 0

paths = needs_xmp["ABS_PATH"].tolist()

if not paths:
    print("✅ All files already have XMP data - nothing to process!")
else:
    print(f"🔄 Starting XMP extraction for {len(paths)} files in batches of {BATCH}...")
    total = len(paths)
    
    for i in range(0, total, BATCH):
        batch = paths[i:i+BATCH]
        js = run_exiftool_json(batch)
        
        for row in js:
            rec = {
                "SourceFile": row.get("SourceFile"),
                "DocumentID": row.get("XMP-xmpMM:DocumentID"),
                "InstanceID": row.get("XMP-xmpMM:InstanceID"),
                "OriginalDocumentID": row.get("XMP-xmpMM:OriginalDocumentID"),
                "DerivedFromDocumentID": row.get("XMP-xmpMM:DerivedFromDocumentID"),
                "DerivedFromInstanceID": row.get("XMP-xmpMM:DerivedFromInstanceID"),
                "FileModifyDate": row.get("File:FileModifyDate"),
            }
            records.append(rec)
        
        print(f"   {min(i+BATCH, total)}/{total}")

# DataFrame erstellen
xmp_df = pd.DataFrame(records)
print("Gelesen:", len(xmp_df))

# Indexe zum späteren Matchen aufbauen:
# - nach DocumentID und nach InstanceID
id_index_doc = (xmp_df[~xmp_df["DocumentID"].isna()]
                 .drop_duplicates(subset=["DocumentID"])
                 .set_index("DocumentID")["SourceFile"].to_dict())

id_index_inst = (xmp_df[~xmp_df["InstanceID"].isna()]
                 .drop_duplicates(subset=["InstanceID"])
                 .set_index("InstanceID")["SourceFile"].to_dict())

xmp_df.head(3)

100/6487
200/6487
300/6487
400/6487
500/6487
600/6487
700/6487
800/6487
900/6487
1000/6487
1100/6487
1200/6487
1300/6487
1400/6487
1500/6487
1600/6487
1700/6487
1800/6487
1900/6487
2000/6487
2100/6487
2200/6487
2300/6487
2400/6487
2500/6487
2600/6487
2700/6487
2800/6487
2900/6487
3000/6487
3100/6487
3200/6487
3300/6487
3400/6487
3500/6487
3600/6487
3700/6487
3800/6487
3900/6487
4000/6487
4100/6487
4200/6487
4300/6487
4400/6487
4500/6487
4600/6487
4700/6487
4800/6487
4900/6487
5000/6487
5100/6487
5200/6487
5300/6487
5400/6487
5500/6487
5600/6487
5700/6487
5800/6487
5900/6487
6000/6487
6100/6487
6200/6487
6300/6487
6400/6487
6487/6487
Gelesen: 6487


,SourceFile,DocumentID,InstanceID,OriginalDocumentID,DerivedFromDocumentID,DerivedFromInstanceID,FileModifyDate
0,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1,uuid:A53BA05DE9CFDA118C30CF8EA9BFE8B1,None,uuid:18FD04FBB2CFDA118C30CF8EA9BFE8B1,uuid:B0B29CD2B8CFDA118C30CF8EA9BFE8B1,None
1,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:12A3774BF0CFDA1186109B3BE9744C33,uuid:53c3d276-af86-430f-b52e-796f0c6a18b8,None,uuid:38614f61-7afe-4454-81d2-8b17e2df8c8f,uuid:07d991a4-40b9-46aa-bebf-1070723b599b,None
2,/home/renku/work/dcaonnextcloud-500gb/DigitalM...,uuid:C74E838EEACFDA118C30CF8EA9BFE8B1,uuid:C84E838EEACFDA118C30CF8EA9BFE8B1,None,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1,uuid:C54E838EEACFDA118C30CF8EA9BFE8B1,None


Bestehendes RDF laden & Graph vorbereiten

In [ ]:
# RDF Graph für XMP-Anreicherung verwenden (von Zelle 5 geladen)
# Verwende den bereits geladenen 'graph' von der Pipeline-Integration
g = graph  # Verwende den bereits geladenen Graph

# Bindings hinzufügen
g.bind("dca", DCA)
g.bind("dca-id", DCA_ID)
g.bind("dca-tech", DCA_TECH)
g.bind("dcterms", DCTERMS)
g.bind("premis", PREMIS)
g.bind("rico", RICO)
g.bind("xsd", XSDNS)

print(f"🔄 Graph übernommen von Pipeline-Integration")
print(f"📊 Tripel (vorher XMP-Anreicherung): {len(g):,}")

Bestehendes RDF geladen: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_DROIDresults.ttl
Tripel (vorher): 94173


XMP‑Identifier als PREMIS‑Identifier anreichern

In [ ]:
# Wir hängen XMP-IDs als PREMIS-Identifier an jede Datei (falls vorhanden).
# ✨ NEU: Verbindung über dca-id:file_<md5[:16]> statt Pfad-Hash

added_ids = 0
md5_based_uris = 0
path_based_uris = 0

for _, row in xmp_df.iterrows():
    abs_path = row["SourceFile"]
    
    # ✨ Neue MD5-basierte URI-Generierung
    file_uri = dca_file_uri_from_path(abs_path)
    
    # Statistik sammeln
    if md5_for_abs_path(abs_path):
        md5_based_uris += 1
    else:
        path_based_uris += 1

    # Sicherstellen, dass Objekt typisiert ist (idR schon vorhanden – doppelte Tripel sind ok)
    g.add((file_uri, RDF.type, DCA.ArchiveFile))
    g.add((file_uri, RDF.type, PREMIS.Object))
    g.add((file_uri, RDF.type, RICO.Record))

    # XMP IDs als PREMIS Identifier
    if row.get("DocumentID"):
        add_identifier_triple(g, file_uri, "XMP DocumentID", row["DocumentID"]); added_ids += 1
    if row.get("InstanceID"):
        add_identifier_triple(g, file_uri, "XMP InstanceID", row["InstanceID"]); added_ids += 1
    if row.get("OriginalDocumentID"):
        add_identifier_triple(g, file_uri, "XMP OriginalDocumentID", row["OriginalDocumentID"]); added_ids += 1

print("✨ Neue ID-Methode erfolgreich verwendet:")
print(f"   MD5-basierte URIs: {md5_based_uris}")
print(f"   Pfad-basierte URIs (Fallback): {path_based_uris}")
print(f"   Identifier-Knoten hinzugefügt: {added_ids}")
print(f"   Tripel (jetzt): {len(g)}")

Identifier-Knoten hinzugefügt: 3543
Tripel (jetzt): 124263


Derivations‑Beziehungen (PREMIS) aus XMP:DerivedFrom ableiten

In [ ]:
added_rel = 0
unresolved = []

for _, row in xmp_df.iterrows():
    child_path = row["SourceFile"]
    child_uri  = dca_file_uri_from_path(child_path)

    parent_path = None
    # 1) Match via DerivedFromDocumentID
    if row.get("DerivedFromDocumentID") and row["DerivedFromDocumentID"] in id_index_doc:
        parent_path = id_index_doc[row["DerivedFromDocumentID"]]
    # 2) sonst via DerivedFromInstanceID
    elif row.get("DerivedFromInstanceID") and row["DerivedFromInstanceID"] in id_index_inst:
        parent_path = id_index_inst[row["DerivedFromInstanceID"]]

    if parent_path:
        parent_uri = dca_file_uri_from_path(parent_path)
        # Tripel hinzufügen (beide Richtungen)
        g.add((child_uri, PREMIS.hasSource, parent_uri))
        g.add((parent_uri, PREMIS.isSourceOf, child_uri))
        added_rel += 1
    else:
        # notieren, wenn eine Ableitung deklariert ist, aber wir kein Gegenstück finden
        if row.get("DerivedFromDocumentID") or row.get("DerivedFromInstanceID"):
            unresolved.append({
                "child": child_path,
                "FromDocID": row.get("DerivedFromDocumentID"),
                "FromInstID": row.get("DerivedFromInstanceID"),
            })

print("PREMIS-Ableitungen erzeugt:", added_rel)
print("Nicht auflösbare Ableitungen:", len(unresolved))
unresolved[:3]

PREMIS-Ableitungen erzeugt: 32
Nicht auflösbare Ableitungen: 27


[{'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/testtexture.jpg',
  'FromDocID': 'uuid:18FD04FBB2CFDA118C30CF8EA9BFE8B1',
  'FromInstID': 'uuid:B0B29CD2B8CFDA118C30CF8EA9BFE8B1'},
 {'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_1/060419_plan1_v03.pdf',
  'FromDocID': 'uuid:38614f61-7afe-4454-81d2-8b17e2df8c8f',
  'FromInstID': 'uuid:07d991a4-40b9-46aa-bebf-1070723b599b'},
 {'child': '/home/renku/work/dcaonnextcloud-500gb/DigitalMaterialCopies/WeingutGantenbein/gramazio-kohler-archiv-server/036_WeingutGantenbein/03_Plaene/01_dFab/060419_Praesentation/060419_Plan_2/060418_Schliessvariante_3.33Seite_12Steine_28Lagen_working_Expression.5.psd',
  'FromDocID': 'uuid:C54E838EEACFDA118C30CF8EA9BFE8B1',
  'F

RDF serialisieren (ergänzte Fassung)

In [ ]:
g.serialize(destination=str(OUTPUT_RDF_PATH), format="turtle")
print("✔ RDF gespeichert:", OUTPUT_RDF_PATH)
print("Tripel gesamt:", len(g))

✔ RDF gespeichert: /home/renku/work/dcaonnextcloud-500gb/dca-metadataraw/WeingutGantenbein/gramazio-kohler-archiv-server_results/gramazio-kohler-archiv-server_catalog_prov.ttl
Tripel gesamt: 124327


SPARQL‑Beispiele (im Graphen) & einfache Visualisierung

In [ ]:
# 1) Alle Derivationen (Child -> Parent)
q1 = """
PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
SELECT ?child ?parent WHERE {
  ?child premis:hasSource ?parent .
} LIMIT 50
"""
for row in g.query(q1):
    print(row.child, " <--derived-- ", row.parent)

# 2) Alle Objekte mit XMP DocumentID
q2 = """
PREFIX premis: <http://www.loc.gov/premis/rdf/v3/>
SELECT ?obj ?val WHERE {
  ?obj premis:hasIdentifier ?id .
  ?id premis:identifierType "XMP DocumentID" ;
      premis:identifierValue ?val .
} LIMIT 50
"""
list(g.query(q2))[:5]

http://dca.ethz.ch/id/file_4d96e0aefb434b6aac962086dcf0e4f57601a15d  <--derived--  http://dca.ethz.ch/id/file_d89f18a520f1f18a0c95c736f0e77cae29c0465b
http://dca.ethz.ch/id/file_fafde06787ea37a75659cfcd89a44e052995bf07  <--derived--  http://dca.ethz.ch/id/file_301a7a04a505ef4f77f28a2325bc639a7ef930f0
http://dca.ethz.ch/id/file_a96f57e0cff78404e1ace2664f8318a28dee350c  <--derived--  http://dca.ethz.ch/id/file_c372ae97eff87606a9ba181560cc681a8276f038
http://dca.ethz.ch/id/file_59e225e743ff43a894b335d3d2f5186da3994d2b  <--derived--  http://dca.ethz.ch/id/file_d956de8e85ebcba108d0ca9526210f638109b579
http://dca.ethz.ch/id/file_3eea00c0303876c27ca536ad3777d65cea80a4c6  <--derived--  http://dca.ethz.ch/id/file_59e225e743ff43a894b335d3d2f5186da3994d2b
http://dca.ethz.ch/id/file_6fcb9b8efedc0042595e06f7007edcd34abac5cb  <--derived--  http://dca.ethz.ch/id/file_3eea00c0303876c27ca536ad3777d65cea80a4c6
http://dca.ethz.ch/id/file_b4322649282885172a2183e1f21e25c20d6e0949  <--derived--  http://dca.

[(rdflib.term.URIRef('http://dca.ethz.ch/id/file_ba9a8c9b2cf17d3a707e2727792c8c7acfdaedec'),
  rdflib.term.Literal('uuid:D9170E74C1CFDA118C30CF8EA9BFE8B1')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_b0a49b8d769bbd7d477c90b3d8da927d5bfeda11'),
  rdflib.term.Literal('uuid:12A3774BF0CFDA1186109B3BE9744C33')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_baadef14ec9dbb98a6ffb0f9a77d4e4d19a2dbe9'),
  rdflib.term.Literal('uuid:C74E838EEACFDA118C30CF8EA9BFE8B1')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_3c2f23ca113671b32fd9581857f5a6eea5648430'),
  rdflib.term.Literal('uuid:83FA0ACCECCFDA118F56F7E076900BAC')),
 (rdflib.term.URIRef('http://dca.ethz.ch/id/file_2e07589df1da1caba87335d6cb2b375c46351974'),
  rdflib.term.Literal('uuid:96dab556-46bb-46d6-9077-e5f9d99616c4'))]

Mapping RDF‑URI → Dateiname/Tooltip

In [ ]:
from pathlib import Path

# --- 1) Map aus XMP-Ergebnissen (bevorzugt, weil garantiert aktueller Pfad) ---
uri_to_label = {}
uri_to_title = {}

if not xmp_df.empty:
    for _, r in xmp_df.iterrows():
        src = r.get("SourceFile")
        if not src:
            continue
        file_uri = dca_file_uri_from_path(src)
        uri_to_label[str(file_uri)] = Path(src).name                # z.B. "testtexture.jpg"
        uri_to_title[str(file_uri)] = src                           # Tooltip: voller Pfad

# --- 2) Fallback: aus Kandidaten-DataFrame (cand) ---
# cand enthält: ABS_PATH, title, identifier
for _, r in cand.iterrows():
    abs_path = r["ABS_PATH"]
    file_uri = dca_file_uri_from_path(abs_path)
    uri_s = str(file_uri)
    if uri_s not in uri_to_label:
        uri_to_label[uri_s] = r.get("title", Path(abs_path).name)    # "title" aus RDF, sonst aus Pfad
    if uri_s not in uri_to_title:
        uri_to_title[uri_s] = abs_path

# --- 3) Sicherheits-Fallback (sollte kaum noch greifen) ---
def label_for(uri: str) -> str:
    return uri_to_label.get(uri, uri.split("/")[-1])  # allerletztes Segment der URI

def title_for(uri: str) -> str:
    return uri_to_title.get(uri, uri)                 # Tooltip im Zweifel die URI selbst

Bonus: Graphische Darstellung der Ableitungs‑Kanten (PREMIS)

In [ ]:
# --- Bonus: Graphische Darstellung der Ableitungs-Kanten (PREMIS) mit Dateinamen-Labels ---
import networkx as nx

# DiGraph Parent -> Child (isSourceOf-Richtung)
G = nx.DiGraph()
for (s, p, o) in g.triples((None, PREMIS.hasSource, None)):
    # s = child, o = parent; Kante Parent -> Child
    G.add_edge(str(o), str(s))

print("Knoten:", G.number_of_nodes(), "Kanten:", G.number_of_edges())

# Versuche pyvis für HTML-Visualisierung
try:
    from pyvis.network import Network
    net = Network(height="700px", width="100%", directed=True, notebook=True)
    net.toggle_physics(True)

    # Knoten mit Dateinamen als Label + vollem Pfad als Tooltip
    for n in G.nodes:
        net.add_node(
            n,
            label=label_for(n),          # z. B. "testtexture.jpg"
            title=title_for(n),          # Tooltip: absoluter Pfad
            shape="dot",
            size=12
        )

    # Kanten mit sprechender Beschriftung
    for u, v in G.edges:
        net.add_edge(u, v, title="premis:isSourceOf / premis:hasSource", arrows="to")

    net.show("derivations_graph.html")
    print("✔ Visualisierung gespeichert: derivations_graph.html")
except Exception as e:
    print("Hinweis: Für HTML-Graph bitte 'pyvis' installieren. Fehler:", e)

Knoten: 49 Kanten: 32
derivations_graph.html
✔ Visualisierung gespeichert: derivations_graph.html


## ✅ Export der finalen erweiterten RDF

**Pipeline-Integration:** Hier exportieren wir die mit XMP-Daten angereicherte RDF zurück für weitere ETH DCA Verarbeitung.

In [ ]:
# =====================================================
# PIPELINE FINALISIERUNG: ERWEITERTE RDF EXPORTIEREN
# =====================================================

# 1. Finale Statistiken vor Export
final_triples = len(graph)
added_triples = final_triples - original_triples
print(f"📈 Final Pipeline Statistics:")
print(f"   Original triples: {original_triples:,}")
print(f"   Final triples:    {final_triples:,}")
print(f"   Added triples:    {+added_triples:,} ({added_triples/original_triples*100:+.1f}%)")

# 2. Export der erweiterten RDF 
print(f"💾 Exporting enriched RDF to: {OUTPUT_RDF_PATH}")
graph.serialize(destination=OUTPUT_RDF_PATH, format='turtle')
exported_size = OUTPUT_RDF_PATH.stat().st_size / 1024 / 1024
print(f"✅ Exported: {exported_size:.2f} MB")

# 3. Validation Export
try:
    # Test-Load der exportierten RDF
    test_graph = Graph()
    test_graph.parse(OUTPUT_RDF_PATH, format='turtle')
    test_triples = len(test_graph)
    print(f"✅ Export validation: {test_triples:,} triples loaded successfully")
    
    if test_triples == final_triples:
        print("✅ Pipeline ERFOLGREICH! RDF bereit für ETH DCA Integration")
    else:
        print(f"⚠️  Mismatch: Expected {final_triples}, got {test_triples}")
        
except Exception as e:
    print(f"❌ Export validation failed: {e}")

# 4. Pipeline-Zusammenfassung
print(f"\n🎯 PIPELINE OUTPUTS:")
print(f"   📥 Input:     {INPUT_RDF_PATH} ({BACKUP_RDF_PATH.stat().st_size/1024/1024:.1f}MB)")
print(f"   💾 Backup:    {BACKUP_RDF_PATH}")
print(f"   📤 Output:    {OUTPUT_RDF_PATH} ({exported_size:.1f}MB)")
print(f"   📊 Processed: {len(cand)} candidate files")
print(f"   ✅ Pipeline:  RenkuLab → XMP → ETH DCA ready")